In [451]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [452]:
data = 'https://raw.githubusercontent.com/Dimassaputra5/DataQuest/refs/heads/main/training_dataset.csv'
data_validation = 'https://raw.githubusercontent.com/Dimassaputra5/DataQuest/refs/heads/main/validation_set.csv'
cc = pd.read_csv(data, 
                 encoding='utf-8',  # atau 'latin1', 'iso-8859-1', dll
                 sep=',')
cc_validation = pd.read_csv(data_validation, 
                 encoding='utf-8',  # atau 'latin1', 'iso-8859-1', dll
                 sep=',')
cc

,customer_number,usia,pekerjaan,status_perkawinan,pendidikan,gagal_bayar_sebelumnya,pinjaman_rumah,pinjaman_pribadi,jenis_kontak,bulan_kontak_terakhir,...,hari_sejak_kontak_sebelumnya,jumlah_kontak_sebelumnya,hasil_kampanye_sebelumnya,tingkat_variasi_pekerjaan,indeks_harga_konsumen,indeks_kepercayaan_konsumen,suku_bunga_euribor_3bln,jumlah_pekerja,pulau,berlangganan_deposito
0,531036,63,sosial media specialis,menikah,Pendidikan Tinggi,no,yes,no,cellular,jul,...,999,0,nonexistent,-1.7,94.215,-40.3,0.885,4991.6,Papua,1
1,999241,43,teknisi,menikah,Pendidikan Tinggi,no,yes,no,cellular,nov,...,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,Sulawesi,0
2,995002,29,sosial media specialis,lajang,Pendidikan Tinggi,no,yes,yes,cellular,jul,...,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,Papua,0
3,932750,40,pekerja kasar,menikah,SMA,no,no,no,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.859,5191.0,Sumatera,1
4,684699,40,sosial media specialis,lajang,Pendidikan Tinggi,no,no,no,cellular,aug,...,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,Bali,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22911,680377,32,teknisi,menikah,Diploma,no,yes,no,cellular,aug,...,999,0,nonexistent,1.4,93.444,-36.1,4.967,5228.1,Kalimantan,0
22912,505429,30,mahasiswa,lajang,Diploma,no,no,no,telephone,sep,...,999,0,nonexistent,-1.1,94.199,-37.5,0.880,4963.6,Bali,0
22913,308952,29,manajer,lajang,SMA,no,yes,no,cellular,jul,...,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,NTT,0
22914,865813,52,entrepreneur,menikah,Tidak Tamat SD,unknown,yes,no,telephone,jun,...,999,0,nonexistent,1.4,94.465,-41.8,4.960,5228.1,NTB,0


In [453]:
cc

,customer_number,usia,pekerjaan,status_perkawinan,pendidikan,gagal_bayar_sebelumnya,pinjaman_rumah,pinjaman_pribadi,jenis_kontak,bulan_kontak_terakhir,...,hari_sejak_kontak_sebelumnya,jumlah_kontak_sebelumnya,hasil_kampanye_sebelumnya,tingkat_variasi_pekerjaan,indeks_harga_konsumen,indeks_kepercayaan_konsumen,suku_bunga_euribor_3bln,jumlah_pekerja,pulau,berlangganan_deposito
0,531036,63,sosial media specialis,menikah,Pendidikan Tinggi,no,yes,no,cellular,jul,...,999,0,nonexistent,-1.7,94.215,-40.3,0.885,4991.6,Papua,1
1,999241,43,teknisi,menikah,Pendidikan Tinggi,no,yes,no,cellular,nov,...,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,Sulawesi,0
2,995002,29,sosial media specialis,lajang,Pendidikan Tinggi,no,yes,yes,cellular,jul,...,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,Papua,0
3,932750,40,pekerja kasar,menikah,SMA,no,no,no,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.859,5191.0,Sumatera,1
4,684699,40,sosial media specialis,lajang,Pendidikan Tinggi,no,no,no,cellular,aug,...,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,Bali,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22911,680377,32,teknisi,menikah,Diploma,no,yes,no,cellular,aug,...,999,0,nonexistent,1.4,93.444,-36.1,4.967,5228.1,Kalimantan,0
22912,505429,30,mahasiswa,lajang,Diploma,no,no,no,telephone,sep,...,999,0,nonexistent,-1.1,94.199,-37.5,0.880,4963.6,Bali,0
22913,308952,29,manajer,lajang,SMA,no,yes,no,cellular,jul,...,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,NTT,0
22914,865813,52,entrepreneur,menikah,Tidak Tamat SD,unknown,yes,no,telephone,jun,...,999,0,nonexistent,1.4,94.465,-41.8,4.960,5228.1,NTB,0


In [454]:
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22916 entries, 0 to 22915
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_number               22916 non-null  int64  
 1   usia                          22916 non-null  int64  
 2   pekerjaan                     22916 non-null  object 
 3   status_perkawinan             22916 non-null  object 
 4   pendidikan                    22916 non-null  object 
 5   gagal_bayar_sebelumnya        22916 non-null  object 
 6   pinjaman_rumah                22916 non-null  object 
 7   pinjaman_pribadi              22916 non-null  object 
 8   jenis_kontak                  22916 non-null  object 
 9   bulan_kontak_terakhir         22916 non-null  object 
 10  hari_kontak_terakhir          22916 non-null  object 
 11  jumlah_kontak_kampanye_ini    22916 non-null  int64  
 12  hari_sejak_kontak_sebelumnya  22916 non-null  int64  
 13  j

In [455]:
cc['status_perkawinan'].unique()
cc['status_perkawinan'] = cc['status_perkawinan'].map({'menikah': 1, 'lajang':2, 'cerai': 3, 'unknown': 4})
cc['status_perkawinan'].unique()
##########################################################
cc_validation['status_perkawinan'] = cc_validation['status_perkawinan'].map({'menikah': 1, 'lajang':2, 'cerai': 3, 'unknown': 4})


array([1, 2, 3, 4])

In [456]:
cc['RasioPekerjaSukuBunga'] = cc['jumlah_pekerja']/cc['suku_bunga_euribor_3bln']
###################################################
cc_validation['RasioPekerjaSukuBunga'] = cc_validation['jumlah_pekerja']/cc_validation['suku_bunga_euribor_3bln']

In [457]:
cc['gagal_bayar_sebelumnya'].unique()
cc['gagal_bayar_sebelumnya'] = cc['gagal_bayar_sebelumnya'].map({'no': 0, 'yes': 1, 'unknown': 2})
cc['gagal_bayar_sebelumnya'].unique()
###############################################
cc_validation['gagal_bayar_sebelumnya'] = cc_validation['gagal_bayar_sebelumnya'].map({'no': 0, 'yes': 1, 'unknown': 2})


array([0, 2, 1])

In [458]:
cc['pinjaman_rumah'].unique()
cc['pinjaman_rumah'] = cc['pinjaman_rumah'].map({'no': 0, 'yes': 1, 'unknown': 2})
cc['pinjaman_rumah'].unique()
##############################################
cc_validation['pinjaman_rumah'] = cc_validation['pinjaman_rumah'].map({'no': 0, 'yes': 1, 'unknown': 2})


array([1, 0, 2])

In [459]:
cc['pinjaman_pribadi'].unique()
cc['pinjaman_pribadi'] = cc['pinjaman_pribadi'].map({'no': 0, 'yes': 1, 'unknown': 2})
cc['pinjaman_pribadi'].unique()
################################################
cc_validation['pinjaman_pribadi'] = cc_validation['pinjaman_pribadi'].map({'no': 0, 'yes': 1, 'unknown': 2})


array([0, 1, 2])

In [460]:
cc['jenis_kontak'].unique()
cc['jenis_kontak'] = cc['jenis_kontak'].map({'cellular': 1, 'telephone': 2})
cc['jenis_kontak'].unique()

array([1, 2])

In [461]:
cc['bulan_kontak_terakhir'].unique()
cc['bulan_kontak_terakhir'] = cc['bulan_kontak_terakhir'].map({
    'jul': 7, 'nov': 11, 'may': 5, 'aug': 8, 'oct': 10, 'apr': 4, 'jun': 6, 'sep': 9, 'mar': 3, 'dec':12
}).astype(int)
cc['bulan_kontak_terakhir'].unique()

array([ 7, 11,  5,  8, 10,  4,  6,  9,  3, 12])

In [462]:
cc['hari_kontak_terakhir'].unique()
cc['hari_kontak_terakhir'] = cc['hari_kontak_terakhir'].map({
    'fri': 5, 'thu': 4, 'wed': 3, 'tue': 2, 'mon': 7})
cc['hari_kontak_terakhir'].unique()

array([5, 4, 3, 2, 7])

In [463]:
cc['hasil_kampanye_sebelumnya'].unique()
cc['hasil_kampanye_sebelumnya'] = cc['hasil_kampanye_sebelumnya'].map({
    'nonexistent': 0, 'failure': 1, 'success': 2
})

In [464]:
cc.select_dtypes(include='object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22916 entries, 0 to 22915
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pekerjaan   22916 non-null  object
 1   pendidikan  22916 non-null  object
 2   pulau       22916 non-null  object
dtypes: object(3)
memory usage: 537.2+ KB


In [465]:
cc = pd.get_dummies(cc, columns=['pekerjaan', 'pendidikan', 'pulau'], drop_first=True)
cc.info()
cc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22916 entries, 0 to 22915
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   customer_number                   22916 non-null  int64  
 1   usia                              22916 non-null  int64  
 2   status_perkawinan                 22916 non-null  int64  
 3   gagal_bayar_sebelumnya            22916 non-null  int64  
 4   pinjaman_rumah                    22916 non-null  int64  
 5   pinjaman_pribadi                  22916 non-null  int64  
 6   jenis_kontak                      22916 non-null  int64  
 7   bulan_kontak_terakhir             22916 non-null  int64  
 8   hari_kontak_terakhir              22916 non-null  int64  
 9   jumlah_kontak_kampanye_ini        22916 non-null  int64  
 10  hari_sejak_kontak_sebelumnya      22916 non-null  int64  
 11  jumlah_kontak_sebelumnya          22916 non-null  int64  
 12  hasi

,customer_number,usia,status_perkawinan,gagal_bayar_sebelumnya,pinjaman_rumah,pinjaman_pribadi,jenis_kontak,bulan_kontak_terakhir,hari_kontak_terakhir,jumlah_kontak_kampanye_ini,...,pendidikan_TIDAK SEKOLAH,pendidikan_Tidak Tamat SD,pendidikan_unknown,pulau_Jawa,pulau_Kalimantan,pulau_NTB,pulau_NTT,pulau_Papua,pulau_Sulawesi,pulau_Sumatera
0,531036,63,1,0,1,0,1,7,5,2,...,False,False,False,False,False,False,False,True,False,False
1,999241,43,1,0,1,0,1,11,5,2,...,False,False,False,False,False,False,False,False,True,False
2,995002,29,2,0,1,1,1,7,4,1,...,False,False,False,False,False,False,False,True,False,False
3,932750,40,1,0,0,0,2,5,3,2,...,False,False,False,False,False,False,False,False,False,True
4,684699,40,2,0,0,0,1,8,3,3,...,False,False,False,False,False,False,False,False,False,False


# Modeling

In [466]:
X = 

SyntaxError: invalid syntax (1712399413.py, line 1)